In [1]:
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator, DataStructs
from rdkit import RDLogger
import csv
import os
import pandas as pd

RDLogger.DisableLog('rdApp.*')

input_files = [
    "smiles_antocianinas.txt",
    "smiles_clorofilas.txt",
    "smiles_flavonoides.txt",
    "smiles_xantofilas.txt"
]
input_folder = "/home/angel/Escritorio"
sdf_file = "/home/angel/Escritorio/Jupyter/chebi.sdf"
output_folder = "/home/angel/Escritorio"
similarity_threshold = 0.8

os.makedirs(output_folder, exist_ok=True)

for input_filename in input_files:
    input_path = os.path.join(input_folder, input_filename)
    output_filename = os.path.splitext(input_filename)[0] + ".csv"
    output_path = os.path.join(output_folder, output_filename)

    with open(input_path, "r") as f:
        smiles_list = [line.strip().split() for line in f if line.strip()]

    supplier = Chem.SDMolSupplier(sdf_file)
    morgan_gen = rdFingerprintGenerator.GetMorganGenerator(radius=3, useBondTypes=True, fpSize=2048)

    with open(output_path, "w", newline="") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=["smiles_base", "kegg_id", "chebi_id", "chebi_smiles", "similarity"])
        writer.writeheader()
        total_count = 0

        for base_smiles, kegg_id in smiles_list:
            base_mol = Chem.MolFromSmiles(base_smiles)
            if base_mol is None:
                print(f"[!] SMILES inválido: {base_smiles} ({kegg_id}) en {input_filename}")
                continue
            base_fp = morgan_gen.GetFingerprint(base_mol)

            for mol in supplier:
                if mol is None:
                    continue
                try:
                    chebi_id = mol.GetProp("ChEBI ID") if mol.HasProp("ChEBI ID") else ""
                    chebi_smiles = Chem.MolToSmiles(mol)
                    if not chebi_id or not chebi_smiles:
                        continue
                    fp = morgan_gen.GetFingerprint(mol)
                    sim = DataStructs.TanimotoSimilarity(base_fp, fp)
                    if sim >= similarity_threshold:
                        writer.writerow({
                            "smiles_base": base_smiles,
                            "kegg_id": kegg_id,
                            "chebi_id": chebi_id,
                            "chebi_smiles": chebi_smiles,
                            "similarity": sim
                        })
                        total_count += 1
                except Exception:
                    continue

        print(f"{output_filename}: {total_count} compuestos similares guardados (umbral ≥ {similarity_threshold})")

    # Eliminar duplicados después de guardar el archivo
    df = pd.read_csv(output_path)
    df.drop_duplicates(inplace=True)
    df.to_csv(output_path, index=False)
    print(f"Eliminados duplicados en {output_filename}. Total de filas únicas: {len(df)}")

print("¡Listo! Archivos de salida sin duplicados generados en:", output_folder)

smiles_antocianinas.csv: 203 compuestos similares guardados (umbral ≥ 0.8)
Eliminados duplicados en smiles_antocianinas.csv. Total de filas únicas: 203
smiles_clorofilas.csv: 16 compuestos similares guardados (umbral ≥ 0.8)
Eliminados duplicados en smiles_clorofilas.csv. Total de filas únicas: 16
smiles_flavonoides.csv: 263 compuestos similares guardados (umbral ≥ 0.8)
Eliminados duplicados en smiles_flavonoides.csv. Total de filas únicas: 263
smiles_xantofilas.csv: 315 compuestos similares guardados (umbral ≥ 0.8)
Eliminados duplicados en smiles_xantofilas.csv. Total de filas únicas: 315
¡Listo! Archivos de salida sin duplicados generados en: /home/angel/Escritorio
